In [2]:
#!/usr/bin/env python

import math
import healpy as hp
from classy import Class
from scipy.stats import norm
import numpy as np
import sys
sys.path.insert(1, '../../../Programas_Cosmologia/PySSC/')
sys.path.insert(1, '../../../Programas_Cosmologia/CLASS/')
import PySSC
import ncounts_gauss_SSC
try:
    import gi

    gi.require_version('NumCosmo', '1.0')
    gi.require_version('NumCosmoMath', '1.0')
except:
    pass
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

Ncm.cfg_init()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())


# Initialize the `Cosmological model`

In [4]:
cosmo = Nc.HICosmoDEXcdm()
cosmo.props.H0  =  67.81
cosmo.props.Omegac  =  0.2612
cosmo.props.Omegab =  0.0486
cosmo.props.Omegax = 1 - 0.0486 -  0.2612
cosmo.props.w = -1.0


reion = Nc.HIReionCamb.new () 
prim = Nc.HIPrimPowerLaw.new () 



cosmo.add_submodel (reion)
cosmo.add_submodel (prim)

dist = Nc.Distance.new (2.0)

tf = Nc.TransferFunc.new_from_name ("NcTransferFuncEH")

psml = Nc.PowspecMLTransfer.new (tf)
psml.require_kmin (1.0e-6)
psml.require_kmax (1.0e3)

psf = Ncm.PowspecFilter.new (psml, Ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0 ()
prim.props.n_SA = 0.9660
print(0.82505858,cosmo.sigma8(psf),cosmo.Omega_k0())

old_amplitude = math.exp (prim.props.ln10e10ASA)
prim.props.ln10e10ASA = math.log ((0.82505858/ cosmo.sigma8(psf))**2 * old_amplitude)
print(0.82505858, cosmo.sigma8(psf))

z_obs_bins  = Ncm.Vector.new_array(np.linspace(0.1,1.8,3))
lnM_obs_bins = Ncm.Vector.new_array(np.linspace(np.log(10)*13.0,np.log(10)*15, 2))

0.82505858 0.8900071895091025 -9.085228043836935e-05
0.82505858 0.82505858


# Construct the $S_{ij}$  Matrix

`kernel and masks`

In [8]:
nz       = 400
z_arr    = np.linspace(0,1.9,num=nz+1)[1:]


nbins_T   = z_obs_bins.len() -1


kernels_T = np.zeros((nbins_T,nz))
for i in range(nbins_T):
    zminbin = z_obs_bins.get(i) ; zmaxbin = z_obs_bins.get(i+1) ; Dz = zmaxbin-zminbin
    for iz in range(nz):
        z = z_arr[iz]
        if ((z>zminbin) and (z<=zmaxbin)):
            kernels_T[i,iz] = 1/Dz




NSIDE = 2*512
NPIX = hp.nside2npix(NSIDE)
m = np.zeros(hp.nside2npix(NSIDE))
pix_theta_ecl, pix_phi_ecl = hp.pix2ang(NSIDE, np.arange(NPIX))

#Guaranteed mask
mask1_guaranteed = np.zeros(NPIX)
mask2_guaranteed = np.zeros(NPIX)
mask3_guaranteed = np.zeros(NPIX)

mask1_guaranteed_condition = (pix_phi_ecl > 3.*np.pi/4.) & (pix_phi_ecl < 13.*np.pi/12.) & (pix_theta_ecl < np.pi/2. - 30.*np.pi/180) & (pix_theta_ecl > np.pi/2. - 80.*np.pi/180)
mask2_guaranteed_condition = (pix_phi_ecl > 11.*np.pi/6.) & (pix_theta_ecl < np.pi/2. - 20.*np.pi/180) & (pix_theta_ecl > np.pi/2. - 40.*np.pi/180)
mask3_guaranteed_condition = (pix_phi_ecl < np.pi/4.) & (pix_theta_ecl < np.pi/2. - 20.*np.pi/180) & (pix_theta_ecl > np.pi/2. - 40.*np.pi/180)

mask1_guaranteed[mask1_guaranteed_condition] = 1
mask2_guaranteed[mask2_guaranteed_condition] = 1
mask3_guaranteed[mask3_guaranteed_condition] = 1

mask_guaranteed = mask1_guaranteed + mask2_guaranteed + mask3_guaranteed

#Total mask
mask1_full = np.zeros(NPIX)
mask2_full = np.zeros(NPIX)
mask3_full = np.zeros(NPIX)

mask1_full_condition = (pix_phi_ecl > 2.*np.pi/3.) & (pix_phi_ecl < 3.*np.pi/2.) & (pix_theta_ecl < np.pi/2. - 10.*np.pi/180) & (pix_theta_ecl > np.pi/2. - 80.*np.pi/180)
mask2_full_condition = (pix_phi_ecl > 11.*np.pi/6.) & (pix_theta_ecl < np.pi/2.) & (pix_theta_ecl > np.pi/2. - 45.*np.pi/180)
mask3_full_condition = (pix_phi_ecl < np.pi/4.) & (pix_theta_ecl < np.pi/2.) & (pix_theta_ecl > np.pi/2. - 45.*np.pi/180)

mask1_full[mask1_full_condition] = 1
mask2_full[mask2_full_condition] = 1
mask3_full[mask3_full_condition] = 1

mask_full = mask1_full + mask2_full + mask3_full






[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         1.17647059 1.17647059 1.17647059
 1.17647059 1.17647059 1.17647059 1.17647059 1.17647059 1.17647059
 1.17647059 1.17647059 1.17647059 1.17647059 1.17647059 1.17647059
 1.17647059 1.17647059 1.17647059 1.17647059 1.17647059 1.17647059
 1.17647059 1.17647059 1.17647059 1.17647059 1.17647059 1.17647059
 1.17647059 1.17647059 1.17647059 1.17647059 1.17647059 1.17647059
 1.17647059 1.17647059 1.17647059 1.17647059 1.17647059 1.17647059
 1.17647059 1.17647059 1.17647059 1.17647059 1.17647059 1.17647059
 1.17647059 1.17647059 1.17647059 1.17647059 1.17647059 1.17647059
 1.17647059 1.17647059 1.17647059 1.17647059 1.17647059 1.17647059
 1.17647059 1.17647059 1.17647059 1.17647059 1.17647059 1.17647059
 1.17647059 1.17647059 1.17647059 1.17647059 1.17647059 1.17647059
 1.17647059 1.17647

# Define proxy modelling


In [ ]:
cluster_m = Nc.ClusterMass.new_from_name("NcClusterMassNodist{'lnM-min':<%20.15e>, 'lnM-max':<%20.15e>}" % (math.log(10)*np.log10(1e14),math.log(10)*np.log10(1e16)))
cluster_z = Nc.ClusterRedshift.new_from_name("NcClusterRedshiftNodist{'z-min': <%20.15e>, 'z-max':<%20.15e>}" % (0.25,2))

# Initialize the `ClusterAbundance` object

In [ ]:
mulf = Nc.MultiplicityFuncTinker.new ()
mulf.set_mdef (Nc.MultiplicityFuncMassDef.CRITICAL)
mulf.set_Delta (200.0)

hmf = Nc.HaloMassFunction.new (dist, psf, mulf)
hbias_Tinker = Nc.HaloBiasTinker.new(hmf)
ca = Nc.ClusterAbundance.new(hmf,hbias_Tinker)

# Data model

$f(x_i)$ for the Gaussian distribution $$-2\ln L = \sum_{ij}\left[(f(x_i) - y_i)C^{-1}_{ij}(f(x_j) - y_j)\right] + \dots$$

In [ ]:
mset = Ncm.MSet.new_array([cosmo,cluster_m,cluster_z])


gauss = ncounts_gauss_SSC.ncounts(len = (z_obs_bins.len()-1)* (lnM_obs_bins.len()-1))
gauss.set_z_obs_bins(z_obs_bins)
gauss.set_lnM_obs_bins(lnM_obs_bins)

NameError: name 'S' is not defined

# Creating the fit object and calculating the Fisher Matrix

`Full-Sky`

In [ ]:
print('Full Sky')
guaranteed_area = 2959.1
ca.set_area(guaranteed_area*(np.pi/180)**2)
gauss.set_kernel(kernels_T)
gauss.set_cad(ca)


rng = Ncm.RNG.seeded_new (None, 12)

gauss.do_cov_func(mset, rng)
gauss.resample(mset, rng)
dset = Ncm.Dataset.new()
dset.append_data(gauss)


cosmo.props.Omegac_fit = True
cosmo.props.Omegab_fit    = False
#cosmo.props.w_fit = True
cosmo.props.H0_fit = True
#prim.props.ln10e10ASA_fit = True
mfunc_oa = Ncm.ObjArray.new ()
mfunc_sigma8 = Ncm.MSetFuncList.new ("NcHICosmo:sigma8", psf)
mfunc_oa.add (mfunc_sigma8) 
print (mfunc_sigma8.eval0 (mset))

mset.prepare_fparam_map ()
#gauss.do_cov_func(mset,rng)

lh = Ncm.Likelihood.new (dset)

fit = Ncm.Fit.new (Ncm.FitType.NLOPT, "ln-neldermead", lh, mset,  Ncm.FitGradType.NUMDIFF_FORWARD)


gauss.y.log_vals("y: ", "%.3f", True)
gauss.cov.log_vals("cov:", "%.3f")

fit.log_info()
fit.fisher()
fit.log_covar ()


: 

`Partial-Sky guaranteed`

In [ ]:
print('Partial Sky guaranteed area')
guaranteed_area = 2959.1
ca.set_area(guaranteed_area*(np.pi/180)**2)
gauss.set_kernel(kernels_T)
gauss.set_mask(mask_guaranteed)
gauss.set_cad(ca)

rng = Ncm.RNG.seeded_new (None, 123)
gauss.resample (mset, rng)

dset = Ncm.Dataset.new()
dset.append_data(gauss)

cosmo.props.Omegac_fit = True
cosmo.props.Omegab_fit    = False
#cosmo.props.w_fit = True
#cosmo.props.H0_fit = True
prim.props.ln10e10ASA_fit = True
mfunc_oa = Ncm.ObjArray.new ()
mfunc_sigma8 = Ncm.MSetFuncList.new ("NcHICosmo:sigma8", psf)
mfunc_oa.add (mfunc_sigma8) 
print (mfunc_sigma8.eval0 (mset))

mset.prepare_fparam_map ()
#gauss.do_cov_func(mset)

lh = Ncm.Likelihood.new (dset)

fit = Ncm.Fit.new (Ncm.FitType.NLOPT, "ln-neldermead", lh, mset,  Ncm.FitGradType.NUMDIFF_FORWARD)


gauss.y.log_vals("y: ", "%.3f", True)
gauss.cov.log_vals("cov:", "%.3f")

fit.log_info()
fit.fisher()
fit.log_covar ()


`Partial-Sky Maximal`

In [ ]:
print('Partial Sky maximal area')
maximal_area = 10009.97
ca.set_area(maximal_area*(np.pi/180)**2)
gauss.set_kernel(kernels_T)
gauss.set_mask(mask_full)
gauss.set_cad(ca)

rng = Ncm.RNG.seeded_new (None, 123)
gauss.resample (mset, rng)

dset = Ncm.Dataset.new()
dset.append_data(gauss)


cosmo.props.Omegac_fit = True
cosmo.props.Omegab_fit    = False
#cosmo.props.w_fit = True
#cosmo.props.H0_fit = True
prim.props.ln10e10ASA_fit = True
mfunc_oa = Ncm.ObjArray.new ()
mfunc_sigma8 = Ncm.MSetFuncList.new ("NcHICosmo:sigma8", psf)
mfunc_oa.add (mfunc_sigma8) 
print (mfunc_sigma8.eval0 (mset))

mset.prepare_fparam_map ()
#gauss.do_cov_func(mset)

lh = Ncm.Likelihood.new (dset)

fit = Ncm.Fit.new (Ncm.FitType.NLOPT, "ln-neldermead", lh, mset,  Ncm.FitGradType.NUMDIFF_FORWARD)


gauss.y.log_vals("y: ", "%.3f", True)
gauss.cov.log_vals("cov:", "%.3f")

fit.log_info()
fit.fisher()
fit.log_covar ()